<a href="https://colab.research.google.com/github/Tongwu18/fu-yao/blob/main/Halle_1_RL%20%E6%9E%81%E9%99%90766.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from typing import List, Tuple, Optional, Dict, Any
from dataclasses import dataclass, field
import copy

df = pd.read_excel("111.xlsx")
df

,玻璃索引,最短宽,高2,Count of 玻璃索引
0,1,112.0,35.0,28
1,2,112.0,35.0,13
2,3,180.0,102.0,30
3,4,180.0,102.0,15
4,5,180.0,102.0,8
...,...,...,...,...
696,697,115.0,79.5,1
697,698,114.0,63.0,1
698,699,209.0,64.0,1
699,700,143.0,64.0,15


# 环境


In [ ]:
"""
玻璃货架装箱 V7 - 修复版
"""

import numpy as np
import pandas as pd
import time
from collections import Counter


# 常量 (mm)
SHELF_W = 360.0
SHELF_H = 800.0
SEP = 27.0
GAP = 9.0
MAX_STACK1 = 4
MAX_STACK2 = 2


class Packer:
    def __init__(self, df):
        id_col = next((c for c in ['玻璃索引', 'glass_id', 'id'] if c in df.columns), None)
        w_col = next((c for c in ['最短宽', 'width'] if c in df.columns), None)
        h_col = next((c for c in ['高2', 'height'] if c in df.columns), None)
        cnt_col = next((c for c in ['Count of 玻璃索引', 'count'] if c in df.columns), None)

        self.ids = df[id_col].values.astype(int) if id_col else np.arange(len(df))
        self.W = df[w_col].values.astype(float)
        self.H = df[h_col].values.astype(float)
        self.C = df[cnt_col].values.astype(int)
        self.n = len(self.W)
        self.total_boxes = self.C.sum()

        total_area_m2 = np.sum(self.W * self.H * self.C) / 1e6
        self.theoretical_min = int(np.ceil(total_area_m2 / (SHELF_W * SHELF_H / 1e6)))

        print(f"玻璃种类: {self.n} | 总箱数: {self.total_boxes}")
        print(f"货架: {SHELF_W:.0f}mm × {SHELF_H:.0f}mm")
        print(f"理论最优: {self.theoretical_min} | VBA基准: 799")
        print("=" * 50)

        self.best_order = None

    def pack(self, order):
        """快速装箱，只返回货架数"""
        remaining = self.C.copy()
        # layer: [height, [(glass_idx, stack_count), ...]]
        shelves = [[[0.0, []]]]

        for idx in order:
            gw, gh = self.W[idx], self.H[idx]

            while remaining[idx] > 0:
                placed = False

                for shelf in shelves:
                    # 1. 尝试叠放
                    for li, layer in enumerate(shelf):
                        lh, positions = layer[0], layer[1]
                        for pi, (gidx, stk) in enumerate(positions):
                            if gidx == idx:
                                max_stk = MAX_STACK1 if li == 0 else MAX_STACK2
                                if stk < max_stk:
                                    new_h = gh * (stk + 1)
                                    other_h = sum(shelf[i][0] for i in range(len(shelf)) if i != li)
                                    sep_h = max(0, len(shelf) - 1) * SEP
                                    if new_h <= SHELF_H - other_h - sep_h:
                                        positions[pi] = (gidx, stk + 1)
                                        shelf[li][0] = max(lh, new_h)
                                        remaining[idx] -= 1
                                        placed = True
                                        break
                        if placed:
                            break
                    if placed:
                        break

                    # 2. 现有层新位置
                    for li, layer in enumerate(shelf):
                        lh, positions = layer[0], layer[1]
                        # 正确计算层已用宽度
                        used_w = sum(self.W[g] for g, _ in positions)
                        if len(positions) > 0:
                            used_w += GAP * len(positions)  # 新位置前要加间距

                        if used_w + gw <= SHELF_W:
                            other_h = sum(shelf[i][0] for i in range(len(shelf)) if i != li)
                            sep_h = max(0, len(shelf) - 1) * SEP
                            if gh <= SHELF_H - other_h - sep_h:
                                positions.append((idx, 1))
                                shelf[li][0] = max(lh, gh)
                                remaining[idx] -= 1
                                placed = True
                                break
                    if placed:
                        break

                    # 3. 新层
                    total_h = sum(l[0] for l in shelf) + max(0, len(shelf) - 1) * SEP
                    if total_h + SEP + gh <= SHELF_H:
                        shelf.append([gh, [(idx, 1)]])
                        remaining[idx] -= 1
                        placed = True
                        break

                if not placed:
                    shelves.append([[gh, [(idx, 1)]]])
                    remaining[idx] -= 1

        return len(shelves)

    def pack_full(self, order):
        """完整装箱，返回详细结构"""
        remaining = self.C.copy()
        shelves = [[[0.0, []]]]

        for idx in order:
            gw, gh = self.W[idx], self.H[idx]

            while remaining[idx] > 0:
                placed = False

                for shelf in shelves:
                    for li, layer in enumerate(shelf):
                        lh, positions = layer[0], layer[1]
                        for pi, (gidx, stk) in enumerate(positions):
                            if gidx == idx:
                                max_stk = MAX_STACK1 if li == 0 else MAX_STACK2
                                if stk < max_stk:
                                    new_h = gh * (stk + 1)
                                    other_h = sum(shelf[i][0] for i in range(len(shelf)) if i != li)
                                    sep_h = max(0, len(shelf) - 1) * SEP
                                    if new_h <= SHELF_H - other_h - sep_h:
                                        positions[pi] = (gidx, stk + 1)
                                        shelf[li][0] = max(lh, new_h)
                                        remaining[idx] -= 1
                                        placed = True
                                        break
                        if placed:
                            break
                    if placed:
                        break

                    for li, layer in enumerate(shelf):
                        lh, positions = layer[0], layer[1]
                        used_w = sum(self.W[g] for g, _ in positions)
                        if len(positions) > 0:
                            used_w += GAP * len(positions)

                        if used_w + gw <= SHELF_W:
                            other_h = sum(shelf[i][0] for i in range(len(shelf)) if i != li)
                            sep_h = max(0, len(shelf) - 1) * SEP
                            if gh <= SHELF_H - other_h - sep_h:
                                positions.append((idx, 1))
                                shelf[li][0] = max(lh, gh)
                                remaining[idx] -= 1
                                placed = True
                                break
                    if placed:
                        break

                    total_h = sum(l[0] for l in shelf) + max(0, len(shelf) - 1) * SEP
                    if total_h + SEP + gh <= SHELF_H:
                        shelf.append([gh, [(idx, 1)]])
                        remaining[idx] -= 1
                        placed = True
                        break

                if not placed:
                    shelves.append([[gh, [(idx, 1)]]])
                    remaining[idx] -= 1

        return shelves

    def get_orders(self):
        orders = {}
        orders['area_desc'] = np.argsort(-(self.W * self.H))
        orders['height_desc'] = np.argsort(-self.H)
        orders['width_desc'] = np.argsort(-self.W)
        orders['volume_desc'] = np.argsort(-(self.W * self.H * self.C))
        orders['h_w_desc'] = np.lexsort((-self.W, -self.H))

        h_order = []
        for h in sorted(set(self.H), reverse=True):
            idxs = list(np.where(self.H == h)[0])
            idxs.sort(key=lambda i: -self.W[i] * self.C[i])
            h_order.extend(idxs)
        orders['h_group_wc'] = np.array(h_order)

        h_order2 = []
        for h in sorted(set(self.H), reverse=True):
            idxs = list(np.where(self.H == h)[0])
            idxs.sort(key=lambda i: -self.C[i])
            h_order2.extend(idxs)
        orders['h_group_c'] = np.array(h_order2)

        h_order3 = []
        for h in sorted(set(self.H), reverse=True):
            idxs = list(np.where(self.H == h)[0])
            idxs.sort(key=lambda i: -self.W[i])
            h_order3.extend(idxs)
        orders['h_group_w'] = np.array(h_order3)

        return orders

    def local_search(self, order, max_iter=5000, max_time=60):
        n = len(order)
        current = list(order)
        current_cost = self.pack(current)
        best, best_cost = current.copy(), current_cost

        start = time.time()
        no_improve, temp = 0, 2.0

        for _ in range(max_iter):
            if time.time() - start > max_time or no_improve > 800:
                break

            new = current.copy()
            r = np.random.random()
            if r < 0.5:
                i, j = np.random.randint(0, n, 2)
                new[i], new[j] = new[j], new[i]
            elif r < 0.8:
                i, j = np.random.randint(n), np.random.randint(n)
                v = new.pop(i)
                new.insert(j, v)
            else:
                i, j = sorted(np.random.randint(0, n, 2))
                new[i:j+1] = new[i:j+1][::-1]

            new_cost = self.pack(new)
            delta = new_cost - current_cost

            if delta < 0 or (temp > 0.01 and np.random.random() < np.exp(-delta / temp)):
                current, current_cost = new, new_cost
                if current_cost < best_cost:
                    best, best_cost = current.copy(), current_cost
                    no_improve = 0
                else:
                    no_improve += 1
            else:
                no_improve += 1
            temp *= 0.997

        return best, best_cost

    def statistics(self, order):
        shelves = self.pack_full(order)
        layer_counts = Counter(len(shelf) for shelf in shelves)

        total_used = 0
        total_boxes = 0
        for shelf in shelves:
            for lh, positions in shelf:
                for idx, stk in positions:
                    total_used += self.W[idx] * self.H[idx] * stk
                    total_boxes += stk
        util = total_used / (len(shelves) * SHELF_W * SHELF_H)

        print("\n" + "=" * 50)
        print("货架统计")
        print("=" * 50)
        print(f"总货架数: {len(shelves)}")
        print(f"总箱数: {total_boxes}")
        print(f"平均利用率: {util*100:.1f}%")
        print()
        print("按层数分类:")
        print("-" * 30)
        for n_layers in sorted(layer_counts.keys()):
            count = layer_counts[n_layers]
            pct = count / len(shelves) * 100
            print(f"  {n_layers}层货架: {count:4d} 个 ({pct:5.1f}%)")
        print("-" * 30)

        return shelves, layer_counts

    def export_plan(self, order, filename="装箱方案.xlsx"):
        shelves = self.pack_full(order)

        rows = []
        for si, shelf in enumerate(shelves):
            # 货架总高
            shelf_h = sum(l[0] for l in shelf) + max(0, len(shelf) - 1) * SEP
            # 货架总箱数
            shelf_boxes = sum(stk for _, positions in shelf for _, stk in positions)
            # 货架利用率
            shelf_used = sum(self.W[idx] * self.H[idx] * stk
                           for _, positions in shelf for idx, stk in positions)
            shelf_util = shelf_used / (SHELF_W * SHELF_H) * 100

            for li, (lh, positions) in enumerate(shelf):
                # 层已用宽度 - 正确计算
                layer_w = sum(self.W[idx] for idx, _ in positions)
                if len(positions) > 1:
                    layer_w += GAP * (len(positions) - 1)

                # 层箱数
                layer_boxes = sum(stk for _, stk in positions)

                for pi, (idx, stk) in enumerate(positions):
                    rows.append({
                        '货架号': si + 1,
                        '货架层数': len(shelf),
                        '货架总高mm': round(shelf_h, 1),
                        '货架利用率%': round(shelf_util, 1),
                        '货架箱数': shelf_boxes,
                        '层号': li + 1,
                        '层高mm': round(lh, 1),
                        '层已用宽mm': round(layer_w, 1),
                        '层箱数': layer_boxes,
                        '位置号': pi + 1,
                        '玻璃ID': self.ids[idx],
                        '玻璃宽mm': self.W[idx],
                        '玻璃高mm': self.H[idx],
                        '叠放数': stk
                    })

        df_out = pd.DataFrame(rows)
        df_out.to_excel(filename, index=False)
        print(f"\n已导出: {filename} ({len(rows)} 条记录)")

        return df_out


def train(df, max_time=280):
    start = time.time()
    packer = Packer(df)

    print("阶段1: 排序策略...")
    orders = packer.get_orders()
    results = [(packer.pack(list(o)), n, list(o)) for n, o in orders.items()]
    for cost, name, _ in sorted(results):
        print(f"  {name}: {cost}")

    results.sort()
    best_cost, best_name, best_order = results[0]
    print(f"最佳: {best_name} = {best_cost} | 耗时: {time.time()-start:.1f}s")

    remain = max_time - (time.time() - start)
    if remain > 30:
        print(f"\n阶段2: 局部搜索...")
        search_time = min(remain - 10, 150) / 3
        for i in range(min(3, len(results))):
            cost, name, order = results[i]
            new_order, new_cost = packer.local_search(order, max_iter=8000, max_time=search_time)
            print(f"  {name}: {cost} -> {new_cost}")
            if new_cost < best_cost:
                best_cost, best_order = new_cost, new_order

    packer.best_order = best_order
    shelves, layer_counts = packer.statistics(best_order)

    print(f"\nvs VBA(799): {best_cost - 799:+d}")
    print(f"vs 理论({packer.theoretical_min}): +{best_cost - packer.theoretical_min}")
    print(f"总耗时: {time.time()-start:.1f}s")

    return packer


def export(packer, filename="装箱方案.xlsx"):
    if packer.best_order is None:
        print("请先运行 train()")
        return
    return packer.export_plan(packer.best_order, filename)

#  训练


In [ ]:
packer = train(df)

# 输出示例:
# 货架统计
# ==================================================
# 总货架数: 766
# 按层数分类:
# ------------------------------
#   1层货架:  120 个 (15.7%)
#   2层货架:  450 个 (58.7%)
#   3层货架:  180 个 (23.5%)
#   4层货架:   16 个 ( 2.1%)
# ------------------------------

# 2. 导出Excel
export(packer, "装箱方案.xlsx")

玻璃种类: 701 | 总箱数: 12117
货架: 360mm × 800mm
理论最优: 594 | VBA基准: 799
阶段1: 排序策略...
  area_desc: 766
  volume_desc: 767
  width_desc: 768
  h_group_c: 769
  h_group_wc: 769
  h_group_w: 772
  h_w_desc: 772
  height_desc: 774
最佳: area_desc = 766 | 耗时: 251.8s

货架统计
总货架数: 766
总箱数: 12117
平均利用率: 77.5%

按层数分类:
------------------------------
  2层货架:  239 个 ( 31.2%)
  3层货架:  308 个 ( 40.2%)
  4层货架:  200 个 ( 26.1%)
  5层货架:   12 个 (  1.6%)
  6层货架:    7 个 (  0.9%)
------------------------------

vs VBA(799): -33
vs 理论(594): +172
总耗时: 284.3s

已导出: 装箱方案.xlsx (5280 条记录)


,货架号,货架层数,货架总高mm,货架利用率%,货架箱数,层号,层高mm,层已用宽mm,层箱数,位置号,玻璃ID,玻璃宽mm,玻璃高mm,叠放数
0,1,2,796.0,59.8,7,1,644.0,331.0,5,1,79,161.0,161.0,4
1,1,2,796.0,59.8,7,1,644.0,331.0,5,2,79,161.0,161.0,1
2,1,2,796.0,59.8,7,2,125.0,349.0,2,1,18,170.0,125.0,1
3,1,2,796.0,59.8,7,2,125.0,349.0,2,2,18,170.0,125.0,1
4,2,2,774.0,68.0,8,1,483.0,348.0,5,1,85,161.0,161.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5275,766,6,755.0,62.4,44,5,98.0,331.0,6,4,390,76.0,49.0,1
5276,766,6,755.0,62.4,44,6,46.0,327.0,4,1,371,75.0,46.0,1
5277,766,6,755.0,62.4,44,6,46.0,327.0,4,2,371,75.0,46.0,1
5278,766,6,755.0,62.4,44,6,46.0,327.0,4,3,370,75.0,46.0,1


In [ ]:
result = pd.read_excel("装箱方案.xlsx")
result.head(20)

,货架号,货架层数,货架总高mm,货架利用率%,货架箱数,层号,层高mm,层已用宽mm,层箱数,位置号,玻璃ID,玻璃宽mm,玻璃高mm,叠放数
0,1,2,796.0,59.8,7,1,644.0,331.0,5,1,79,161.0,161.0,4
1,1,2,796.0,59.8,7,1,644.0,331.0,5,2,79,161.0,161.0,1
2,1,2,796.0,59.8,7,2,125.0,349.0,2,1,18,170.0,125.0,1
3,1,2,796.0,59.8,7,2,125.0,349.0,2,2,18,170.0,125.0,1
4,2,2,774.0,68.0,8,1,483.0,348.0,5,1,85,161.0,161.0,3
5,2,2,774.0,68.0,8,1,483.0,348.0,5,2,7,178.0,140.0,2
6,2,2,774.0,68.0,8,2,264.0,351.2,3,1,8,177.2,132.0,2
7,2,2,774.0,68.0,8,2,264.0,351.2,3,2,46,165.0,130.0,1
8,3,2,777.0,60.5,8,1,500.0,356.2,5,1,8,177.2,132.0,1
9,3,2,777.0,60.5,8,1,500.0,356.2,5,2,18,170.0,125.0,4
